# **Document for this file**
## **Notations:**  
1. **E: Evidence.** We use a nested list, consisting chars and lists of chars, to represent the evidence.   
eg: **['A','B', ['A','B'], ['A','B'], ['A','B']]**, standing for: **{ L1 = 'A' & L2 = 'B' & L3 not in {'A','B'} & L4 not in {'A','B'} & L5 not in {'A','B'} }**  
2. **L: Belief, a list of 5 chars**  
eg: **['A','A','0','0','0']**, standing for: **{L1 = 'A' or L2 = 'A'}** 

## **MATH:**  
**P(L|E) = ΣP(L|W)P(E|W)P(W) / ΣP(E|W')P(W')**  

## **Implementation:**  
### (i).Preprocess:  
1. We calculate all the value of P(L|W), P(E|W), P(W), saved in three numpy arrays respectively, namely **PROB_L_W**, **PROB_E_W** and **PROB_W.**  
### (ii).Numpy for fast calculation and avoiding redundant computations:  
1. For **ΣP(L|W)P(E|W)P(W)**: We element-wise multiply **PROB_L_W, PROB_E_W, PROB_W**, and sum up elements in the result array. i.e. **(PROB_L_W * PROB_E_W * PROB_W).sum().item()**  
2. For **ΣP(E|W')P(W')**: We do dot product for **PROB_E_W**, **PROB_W**. i.e. **PROB_E_W.dot(PROB_W).item()**  
### (iii).Inference for the best result:
1. We do a for loop for 26 letters(A->Z), and find the **max{P(letter_i|E)}** for all letters.

# Fetch the data to DataFrame

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('./hw2_word_counts_05.txt',sep=" ", engine='python')
df

# (a) Calculate P(W), saved in df, and anwser question(a)

In [52]:
total_counts = df['Times'].sum()
df['prob_w'] = df['Times'] / total_counts

df.nlargest(10, ['Times'])

,Word,Times,prob_w
5821,THREE,273077,0.035627
5102,SEVEN,178842,0.023333
1684,EIGHT,165764,0.021626
6403,WOULD,159875,0.020858
18,ABOUT,157448,0.020542
5804,THEIR,145434,0.018974
6320,WHICH,142146,0.018545
73,AFTER,110102,0.014365
1975,FIRST,109957,0.014346
1947,FIFTY,106869,0.013943


# (b) Inference for best guesses

## Global Variables

In [53]:
LEN_DF = 6535
WORD_LEN = 5

PROB_L_W = np.zeros((LEN_DF, ))  
PROB_E_W = np.zeros((LEN_DF, ))
PROB_W = np.zeros((LEN_DF, ))

## Renew Tensor P(L | W)

In [54]:
def renew_tensors(L, E):
    for i in range(LEN_DF):
        PROB_L_W[i] = prob_L_given_W(L, df['Word'][i])
        PROB_E_W[i] = prob_E_given_W(E, df['Word'][i])
        PROB_W[i] = df['prob_w'][i]


## Helper Functions:

In [55]:
# P(L | W)  
def prob_L_given_W(L, word):
    '''
    L : ['S', 'S', '0', '0', 'S'], relationship between letters: OR
    word : a string, eg: "AAARR"

    return type: boolean
    '''
    for i in range(5):
        if L[i] == word[i]:
            return 1
    return 0

# P(E | W)  
def prob_E_given_W(E, word):
    '''
    E: evidenvce, a nested list. relationship between elements: AND
    word: a string from corpus
    '''
    for i in range(5):
        if type(E[i]) == str and E[i] != word[i]:
            return 0
        elif type(E[i]) == list:
            for letter in E[i]:
                if word[i] == letter:
                    return 0
    return 1

# Belief Helper Function: L contructor given E
def L_constructor(letter, E):
    '''
    letter: letter we want to guess, a single char
    E: evidence, a nested list
    In this function, we construct the L with given evidence, and return L
    '''
    L = []
    for i in range(5):
        if type(E[i]) == str :
            L.append('0')
        elif type(E[i]) == list:
            L.append(letter)
    return L

# Consturct a list for a specific evidence
def E_contructor(correctly_guessed, incorrectly_guessed):
    '''
    correctly_guessed: a list with 5 char elements. eg: ['A', 'B', '0', '0', '0'], whose '0' stands for still unknown.
    incorrectly_guessed: a set with some elements that won't appear in this word
    return a nested list
    '''
    E = []
    incorrectly_guessed = list(incorrectly_guessed) + [i for i in correctly_guessed if i != '0']
    for i in range(5):
        if correctly_guessed[i] != '0':
            E.append(correctly_guessed[i])
        else:
            E.append(incorrectly_guessed)
    return E

# decode for letter
def get_letter(i):
    return chr(65 + i)

def get_already_known(E):
    '''
    return a list that contains letter we've already used
    '''
    already_known = []
    for element in E:
        if type(element) == list:
            already_known = element
    return already_known


## Predictive Result for L given E: P(L | E)

In [56]:
def predictiveResult(L, E):
    '''
    return predictive result for L given E
    '''
    pred = 0
    prob_E = PROB_E_W.dot(PROB_W).item()
    pred = (PROB_L_W * PROB_E_W * PROB_W).sum().item() / prob_E
    return pred

## Inference for best guess

In [57]:
def best_guess(E):
    '''
    para:
    E: evidence, a nested list 
    It will return result of best guess with the given evidence and corpus
    '''
    best_result = ''
    best_result_prob = 0
    already_known = get_already_known(E)
    
    # for 26 letters from 'A' to 'Z', each 'letter' = 65 + i
    for i in range(26):
        letter = get_letter(i)
        if letter in already_known:
            continue
        L = L_constructor(letter, E)
        # print ("letter ", letter, ": ", end='')
        renew_tensors(L, E)
        pred = predictiveResult(L, E)
        if best_result_prob < pred:
            best_result_prob = pred
            best_result = letter

    return best_result, best_result_prob    

## Calculate all results for (b)

In [58]:
E = E_contructor(['0', '0', '0', '0', '0'], {})
print(best_guess(E))
E = E_contructor(['0', '0', '0', '0', '0'], {'E', 'O'})
print(best_guess(E))
E = E_contructor(['Q', '0', '0', '0', '0'], {})
print(best_guess(E))
E = E_contructor(['Q', '0', '0', '0', '0'], {'U'})
print(best_guess(E))
E = E_contructor(['0', '0', 'Z', 'E', '0'], {'A', 'D', 'I', 'R'})
print(best_guess(E))
E = E_contructor(['0', '0', '0', '0', '0'], {'E', 'O'})
print(best_guess(E))
E = E_contructor(['D', '0', '0', 'I', '0'], {})
print(best_guess(E))
E = E_contructor(['D', '0', '0', 'I', '0'], {'A'})
print(best_guess(E))
E = E_contructor(['0', 'U', '0', '0', '0'], {'A','E','I','O','S'})
print(best_guess(E))

('E', 0.5394172389647974)
('I', 0.6365554141009611)
('U', 0.9866727159303579)
('A', 1.0)
('O', 0.8803418803418803)
('I', 0.6365554141009611)
('A', 0.8206845238095236)
('E', 0.7520746887966805)
('Y', 0.6269651101630529)
